In [58]:
import pandas as pd
import nycflights13 as flights

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

# 항공편 데이터 (main dataset)
df_flights = flights.flights  
df_airlines = flights.airlines 
df_airports = flights.airports 
df_planes = flights.planes 
df_weather = flights.weather 

pd.set_option('display.max_rows', None)      # 모든 행 출력
pd.set_option('display.max_columns', None)

In [2]:
# print(df_flights.info())
# print(df_flights.isnull().sum())
flights_cleaned = df_flights.dropna(subset=['dep_time','dep_delay', 'arr_time','arr_delay','tailnum', 'air_time'])
flights_cleaned = flights_cleaned.drop('year',axis=1)
flights_cleaned.shape

(327346, 18)

In [3]:
cond1 = (flights_cleaned['carrier'] == 'UA') | (flights_cleaned['carrier'] == 'EV') | (flights_cleaned['carrier'] == 'B6')
new_flights_df = flights_cleaned[cond1]
# print(new_df)
# print(new_df.shape) (162939, 19)
print(new_flights_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 162939 entries, 0 to 336769
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   month           162939 non-null  int64  
 1   day             162939 non-null  int64  
 2   dep_time        162939 non-null  float64
 3   sched_dep_time  162939 non-null  int64  
 4   dep_delay       162939 non-null  float64
 5   arr_time        162939 non-null  float64
 6   sched_arr_time  162939 non-null  int64  
 7   arr_delay       162939 non-null  float64
 8   carrier         162939 non-null  object 
 9   flight          162939 non-null  int64  
 10  tailnum         162939 non-null  object 
 11  origin          162939 non-null  object 
 12  dest            162939 non-null  object 
 13  air_time        162939 non-null  float64
 14  distance        162939 non-null  int64  
 15  hour            162939 non-null  int64  
 16  minute          162939 non-null  int64  
 17  time_hour      

In [27]:
df_planes.shape
new_planes_df = df_planes.drop('speed',axis=1)
# new_planes_df.info()
new_planes_df = new_planes_df.dropna(subset=['year'])
# new_planes_df.info() 
# new_planes_df.shape (3252,8)

In [28]:
merged_df = pd.merge(new_flights_df, new_planes_df, 
                      on = 'tailnum', how = 'inner')

merged_df.isnull().sum()
merged_df.info()
# merged_df.shape (157902, 25)
bins = [0, 700, 3000, 5000]
labels = ['단거리', '중거리', '장거리']
merged_df['거리구분'] =  pd.cut(merged_df['distance'], bins=bins, labels=labels)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157902 entries, 0 to 157901
Data columns (total 25 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   month           157902 non-null  int64  
 1   day             157902 non-null  int64  
 2   dep_time        157902 non-null  float64
 3   sched_dep_time  157902 non-null  int64  
 4   dep_delay       157902 non-null  float64
 5   arr_time        157902 non-null  float64
 6   sched_arr_time  157902 non-null  int64  
 7   arr_delay       157902 non-null  float64
 8   carrier         157902 non-null  object 
 9   flight          157902 non-null  int64  
 10  tailnum         157902 non-null  object 
 11  origin          157902 non-null  object 
 12  dest            157902 non-null  object 
 13  air_time        157902 non-null  float64
 14  distance        157902 non-null  int64  
 15  hour            157902 non-null  int64  
 16  minute          157902 non-null  int64  
 17  time_hour 

In [29]:
merged_df.head(10)
merged_df[['distance','거리구분']].head()

,distance,거리구분
0,1400,중거리
1,1416,중거리
2,1576,중거리
3,719,중거리
4,1065,중거리


In [30]:
cond1 = (merged_df['carrier'] == 'UA')
ua = merged_df[cond1]
cond2 = (merged_df['carrier'] == 'EV')
ev = merged_df[cond2]
cond3 = (merged_df['carrier'] == 'B6')
b6 = merged_df[cond3]

In [31]:
print(ua.shape) # 1등  항공사
print(b6.shape) # 2등
print(ev.shape) # 3등

(55929, 26)
(52384, 26)
(49589, 26)


In [32]:
ua['운항_횟수'] = 1
b6['운항_횟수'] = 1
ev['운항_횟수'] = 1

C:\Users\USER\AppData\Local\Temp\ipykernel_2036\203667343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ua['운항_횟수'] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_2036\203667343.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  b6['운항_횟수'] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_2036\203667343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [ ]:
g1_ua = ua.groupby(['manufacturer','year'])['운항_횟수'].count()
g1_ua

# ua 항공사는 1990년대 항공기를 가지고 많이 운행
# BOING 엔진 제조업체 
# 나머지 2등, 3등 회사는 BOEING 회사 쓰지를 않음
# AIRBUS 그룹 엔진을 쓰다가 2002년 이후로 BOEING 엔진만 쓰고 있다
# 왜 2002년 이후 BOEING 엔진만 쓰는가?
# AIRBUS 그룹 엔진 쓰지 않는 이유?


manufacturer      year  
AIRBUS            2002.0    1627
AIRBUS INDUSTRIE  1993.0     496
                  1994.0    1531
                  1995.0     767
                  1996.0     654
                  1997.0     964
                  1998.0    3183
                  1999.0    1188
                  2000.0    1648
                  2001.0    3383
                  2002.0     581
BOEING            1965.0       4
                  1989.0     763
                  1990.0    2146
                  1991.0    1358
                  1992.0    2838
                  1993.0     947
                  1994.0     965
                  1995.0     301
                  1996.0     268
                  1997.0     850
                  1998.0    6146
                  1999.0    5118
                  2000.0    2213
                  2001.0    2901
                  2002.0    1043
                  2003.0     357
                  2004.0    1376
                  2005.0     796
                  

In [ ]:
g2_b6 = b6.groupby(['manufacturer','year'])['운항_횟수'].count()
g2_b6

# b6 항공사는 2000년대 이상의 항공기 연식을 가지고 있음
# 신생 브랜드일 가능성
# 2002년 이후로 AIRBUS 엔진 사용
# EMBRAER도 꾸준히 사용
# AIRBUS INDUSTRIE 2002년 이후로 안쓰다가 2013년식 18번 운항
# 1등 UA는 BOEING 엔진, 2등 B6 AIRBUS, EMBRAER, 3등 EMBRAER, BOMBARDIER INC 

manufacturer            year  
AIRBUS                  2002.0    1452
                        2003.0    3814
                        2004.0    4703
                        2005.0    4554
                        2006.0    4256
                        2007.0    3247
                        2008.0    2998
                        2009.0     812
                        2011.0    1146
                        2012.0    1766
                        2013.0     267
AIRBUS INDUSTRIE        1999.0     549
                        2000.0    1689
                        2001.0    1678
                        2002.0     540
                        2013.0      18
CIRRUS DESIGN CORP      2007.0     289
EMBRAER                 2005.0    2656
                        2006.0    4443
                        2007.0    2238
                        2008.0    1246
                        2009.0    1818
                        2010.0    1308
                        2011.0    1845
                        2012.0   

In [ ]:
g3_ev = ev.groupby(['manufacturer','year'])['운항_횟수'].count()
g3_ev

# BOMBARDIER INC, EMBRAER 이 두 브랜드 많이 사용
# CANADAIR 상대적으로 적음

# B6 와 공통 제조업체 EMBRAER

manufacturer    year  
BOMBARDIER INC  1998.0     689
                1999.0       5
                2000.0       4
                2001.0     676
                2002.0    1803
                2003.0    1790
                2004.0    2425
                2005.0     720
                2006.0      41
                2007.0     161
                2008.0     341
                2009.0     348
CANADAIR        1997.0     957
                1998.0     535
EMBRAER         1998.0    2744
                1999.0    4861
                2000.0    4541
                2001.0    6776
                2002.0    9255
                2003.0    4339
                2004.0    2882
                2005.0    2667
                2006.0    1029
Name: 운항_횟수, dtype: int64

In [13]:
pd.set_option('display.max_rows', None)

# 모든 열 출력
pd.set_option('display.max_columns', None)

In [ ]:
g4_ua = ua.groupby(['manufacturer','year','seats'])[['운항_횟수']].count()
g4_ua

# AIRBUS, AIRBUS INDUSTRIE, BOEING
# AIRBUS 엔진을 탑재한 항공기들은 좌석 수 179, 200 만 있음
# BOEING 엔진을 탑재한 항공기들은 다양한 좌석 수, 최대 400 좌석을 가진 항공기 보유
# 2002년 이후로 AIRBUS 그룹 엔진을 이용한 항공기는 만들지 않고, BOEING 엔진을 탑재한 항공기 사용
# 좌석이 400인 BOEING 엔진을 탑재한 항공기 연삭(1995, 1997, 1998, 1999, 2000, 2002) 이렇게 가지고 있다.


운항_횟수
manufacturer     year   seats       
AIRBUS           2002.0 179      594
                        200     1033
AIRBUS INDUSTRIE 1993.0 200      496
                 1994.0 200     1531
                 1995.0 200      767
                 1996.0 200      654
                 1997.0 179      471
                        200      493
                 1998.0 179     2291
                        200      892
                 1999.0 179      572
                        200      616
                 2000.0 179      629
                        200     1019
                 2001.0 179     1607
                        200     1776
                 2002.0 179      361
                        200      220
BOEING           1965.0 149        4
                 1989.0 178      763
                 1990.0 178     2146
                 1991.0 178     1355
                        330        3
                 1992.0 178     2827
                        330       11
                 1993.0 178      927
                        330       20
                 1994.0 178      965
                 1995.0 178      300
                        400        1
                 1996.0 178      268
                 1997.0 178      849
                        400        1
                 1998.0 149     4517
                        178     1561
                        330       67
                        400        1
                 1999.0 149     4474
                        178      570
                        330       71
                        400        3
                 2000.0 149     1771
                        178      273
                        292      108
                        330       60
                        400        1
                 2001.0 149     1788
                        191      862
                        255       13
                        275      145
                        292       62
                        330       31
                 2002.0 149      405
                        191      152
                        275      127
                        292      358
                        400        1
                 2003.0 149      335
                        275       22
                 2004.0 149     1178
                        275      198
                 2005.0 149      796
                 2006.0 149      463
                 2008.0 149     1194
                        191     1532
                 2009.0 149      120
                        191     1281
                 2010.0 149     1154
                        191      210
                 2011.0 149      251
                        191      122
                 2012.0 191     1925
                        260        6
                 2013.0 191     1259

In [ ]:
g5_b6 = b6.groupby(['manufacturer','year','seats'])[['운항_횟수']].count()
g5_b6

# b6 항공사의 항공기 엔진 제조업체 총 5개
# AIRBUS,AIRBUS INDUSTRIE, CIRRUS DESIGN CORP, EMBRAER, ROBINSON HELICOPTER CO
# UA와 차이점 및 알아볼 만한 거?
# 1. BOEING 엔진 제조업체 사용 X
# 2. AIRBUS 2002년식부터 꾸준히 사용 (UA는 2002년 이후로 AIRBUS 엔진 항공기 사용 X)
# 3. 좌석수 주로 200이 많고, 2013년식 좌석 수 379 항공기 도입 (UA는 2002년 이후로 300석이 넘는 항공기 도입하지 않음)
# 4. EMBRAER 항공기의 좌석수가 20개인 항공기를 2005년 이후로 꾸준히 도입
# 5. 연식이 최신일수록 운항회수가 비례하지 않음
# 6. 2003~2008년식 운항 많음 (모든 제조업체)


운항_횟수
manufacturer           year   seats       
AIRBUS                 2002.0 200     1452
                       2003.0 200     3814
                       2004.0 200     4703
                       2005.0 200     4554
                       2006.0 200     4256
                       2007.0 200     3247
                       2008.0 200     2998
                       2009.0 200      812
                       2011.0 200     1146
                       2012.0 200     1766
                       2013.0 200      232
                              379       35
AIRBUS INDUSTRIE       1999.0 200      549
                       2000.0 200     1689
                       2001.0 200     1678
                       2002.0 200      540
                       2013.0 379       18
CIRRUS DESIGN CORP     2007.0 4        289
EMBRAER                2005.0 20      2656
                       2006.0 20      4443
                       2007.0 20      2238
                       2008.0 20      1246
                       2009.0 20      1818
                       2010.0 20      1308
                       2011.0 20      1845
                       2012.0 20      1139
                       2013.0 20      1632
ROBINSON HELICOPTER CO 2012.0 5        281

In [ ]:
ev_b6 = ev.groupby(['manufacturer','year','seats'])[['운항_횟수']].count()
ev_b6

# EV 항공사
# 엔진 제조업체 3 종류
# BOMBARDIER INC, CANADAIR, EMBRAER
# 모든 좌석 수는 100개 넘는 항공기 없음
# 2009년 이후로 항공기 사지 않았음
# EMBRAER 엔진을 탑재한 항공기의 운항횟수가 많음

# 

운항_횟수
manufacturer   year   seats       
BOMBARDIER INC 1998.0 55       689
               1999.0 55         5
               2000.0 55         4
               2001.0 55       190
                      80       486
               2002.0 55       179
                      80      1624
               2003.0 55         4
                      80      1786
               2004.0 55         1
                      80      2424
               2005.0 80       720
               2006.0 95        41
               2007.0 95       161
               2008.0 95       341
               2009.0 95       348
CANADAIR       1997.0 55       957
               1998.0 55       535
EMBRAER        1998.0 55      2744
               1999.0 55      4861
               2000.0 55      4541
               2001.0 55      6776
               2002.0 55      9255
               2003.0 55      4339
               2004.0 55      2882
               2005.0 55      2667
               2006.0 55      1029

In [17]:
merged_df["distance"].max()
# 단거리 (Short-haul): 약 0~1,500마일 (0~2,400km)

# 중거리 (Medium-haul): 약 1,500마일 ~ 2,500~3,000마일 (2,400~4,000~4,800km)

# 장거리 (Long-haul): 약 3,000마일 이상 (4,800km 이상)

4963

In [ ]:
g7_ua = ua.groupby(['year','거리구분'])['운항_횟수'].count()
g7_ua

# 장거리 운행 2000년~2002년 연식 비행기만 운행
# 1965년식 항공기 단거리 운행 

C:\Users\USER\AppData\Local\Temp\ipykernel_2036\1871974195.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g7_ua = ua.groupby(['year','거리구분'])['운항_횟수'].count()


year    거리구분
1965.0  단거리        4
        중거리        0
        장거리        0
1989.0  단거리       63
        중거리      700
        장거리        0
1990.0  단거리      387
        중거리     1759
        장거리        0
1991.0  단거리      450
        중거리      906
        장거리        2
1992.0  단거리      610
        중거리     2225
        장거리        3
1993.0  단거리      677
        중거리      765
        장거리        1
1994.0  단거리     1635
        중거리      861
        장거리        0
1995.0  단거리      743
        중거리      325
        장거리        0
1996.0  단거리      626
        중거리      296
        장거리        0
1997.0  단거리     1011
        중거리      803
        장거리        0
1998.0  단거리     5915
        중거리     3414
        장거리        0
1999.0  단거리     4155
        중거리     2151
        장거리        0
2000.0  단거리     2502
        중거리     1293
        장거리       66
2001.0  단거리     4249
        중거리     2002
        장거리       33
2002.0  단거리     2083
        중거리      908
        장거리      260
2003.0  단거리      206
        중거리      151


In [ ]:
g8_b6 = b6.groupby(['manufacturer','year','거리구분'])['운항_횟수'].count()
g8_b6

# b6 항공사의 항공기 엔진 제조업체는 총 5개
# AIRBUS, AIRBUS INDUSTRIE, CIRRUS DESIGN CORP, EMBRAER, ROBINSON HELICOPTER CO
# b6 항공사는 장거리 비행 하나도 없음 ***
# AIRBUS 중거리 비행보단 단거리 비행 비율이 높다
# EMBRAER 중거리 단거리 운행, 단거리 비율이  높다.
# 나머지 3개도 단거리 비율이 많음

C:\Users\USER\AppData\Local\Temp\ipykernel_2036\2020532388.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g8_b6 = b6.groupby(['manufacturer','year','거리구분'])['운항_횟수'].count()


manufacturer            year    거리구분
AIRBUS                  1999.0  단거리        0
                                중거리        0
                                장거리        0
                        2000.0  단거리        0
                                중거리        0
                                장거리        0
                        2001.0  단거리        0
                                중거리        0
                                장거리        0
                        2002.0  단거리      982
                                중거리      470
                                장거리        0
                        2003.0  단거리     2561
                                중거리     1253
                                장거리        0
                        2004.0  단거리     3219
                                중거리     1484
                                장거리        0
                        2005.0  단거리     3010
                                중거리     1544
                                장거리        0
                  

In [ ]:
g8_ev = ev.groupby(['manufacturer','year','거리구분'])['운항_횟수'].count()
g8_ev

# ev 항공사의 항공기 엔진 제조업체
# BOMBARDIER INC, CANADAIR, EMBRAER 3개
# ev 항공사는 단거리 운행만 하는 항공사
# EMBRAER 단거리만 운행

C:\Users\USER\AppData\Local\Temp\ipykernel_2036\3589122051.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g8_ev = ev.groupby(['manufacturer','year','거리구분'])['운항_횟수'].count()


manufacturer    year    거리구분
BOMBARDIER INC  1997.0  단거리        0
                        중거리        0
                        장거리        0
                1998.0  단거리      689
                        중거리        0
                        장거리        0
                1999.0  단거리        5
                        중거리        0
                        장거리        0
                2000.0  단거리        4
                        중거리        0
                        장거리        0
                2001.0  단거리      676
                        중거리        0
                        장거리        0
                2002.0  단거리     1803
                        중거리        0
                        장거리        0
                2003.0  단거리     1790
                        중거리        0
                        장거리        0
                2004.0  단거리     2425
                        중거리        0
                        장거리        0
                2005.0  단거리      720
                        중거리        0
         

In [ ]:
## 최종 정리
# UA 항공사(1등)
# 1990년대 항공기를 많이 보유
# 제조사 종류: AIRBUS, AIRBUS INDUSTRIE, BOEING
# 2002년 이후로 AIRBUS 엔진 탑재한 항공기 도입하지 않고, BOEING 엔진 탑재한 항공기만 도입
# 단거리, 중거리, 장거리 모두 다 비행
# 장거리 운행 2000년~2002년 연식 비행기만 운행
# 2000연식 이전 항공기가 2000연식 이후 항공기보다 운항 수가 많다.
# 좌석 수 400개인 BOEING 엔진 탑재한 항공기 보유

# B6 (2등)
# b6 항공사의 항공기 엔진 제조업체는 총 5개
# 제조사 종류: AIRBUS, AIRBUS INDUSTRIE, CIRRUS DESIGN CORP, EMBRAER, ROBINSON HELICOPTER CO
# b6 항공사는 단거리, 중거리 비행
# EMBRAER 항공기의 좌석수가 20개인 항공기를 2005년 이후로 꾸준히 도입
# UA항공사보다 최신식 항공기 사용한다는 점
# 좌석 수 379인 2013년식 항공기 도입하여 운항


# EV (3등)
# EV 항공사의 항공기 엔진 제조업체 총 3개 
# 제조사 종류 BOMBARDIER INC, CANADAIR, EMBRAER
# EV 항공사는 단거리 운행만 하는 항공사
# 모든 좌석 수는 100개 넘는 항공기 없음
# 2009년 이후로 항공기 도입 하지 않았음

# 공통점
# 1. 다양한 연식별로 다양한 엔진 제조사들을 항공기에 도입하여 운항을 함.
# 2. 항공기 연식이 최신일수록 항공기 운항 횟수가 많지 않음. -> 오히려 2000년대 초중반 연식 항공기 운항횟수 많음
# 3. 모든 항공사들 단거리 운행 비율이 높다


# 좌석, 거리,
# 뭘 더 해야 조금 더 의미있는 게 나올까 
# 항공사 별 제조사 별 지연 봐야 ㅗ디겠ㅈ?

In [48]:
cond1 = ev['dep_delay'] > 15
new_ev = ev[cond1][['거리구분', '운항_횟수','seats']]


cond2 = (new_ev['거리구분'] == "단거리")
cond3 = (new_ev['거리구분'] == "중거리")

new_short_ev = new_ev[cond2]["운항_횟수"]
a = new_short_ev.sum() # 10032

new_mid_ev = new_ev[cond3]["운항_횟수"]
b = new_mid_ev.sum() # 5017
# new = ev.groupby(['거리구분','seats'])['운항_횟수'].sum()
# new.reset_index()




In [54]:
c = len(ev[ev["거리구분"] == "단거리"]) # 33560
d = len(ev[ev["거리구분"] == "중거리"]) # 16029

print(round(a/c*100, 4))
print(round(b/d*100,4))



29.8927
31.2995


In [60]:
ev.head(5)

,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour,year,type,manufacturer,model,engines,seats,engine,거리구분,운항_횟수
5,1,1,557.0,600,-3.0,709.0,723,-14.0,EV,5708,N829AS,LGA,IAD,53.0,229,6,0,2013-01-01T11:00:00Z,1998.0,Fixed wing multi engine,CANADAIR,CL-600-2B19,2,55,Turbo-fan,단거리,1
18,1,1,624.0,630,-6.0,909.0,840,29.0,EV,4626,N11107,EWR,MSP,190.0,1008,6,30,2013-01-01T11:00:00Z,2002.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,Turbo-fan,중거리,1
20,1,1,632.0,608,24.0,740.0,728,12.0,EV,4144,N13553,EWR,IAD,52.0,212,6,8,2013-01-01T11:00:00Z,2002.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,Turbo-fan,단거리,1
57,1,1,809.0,815,-6.0,1043.0,1050,-7.0,EV,4388,N14542,EWR,JAX,132.0,820,8,15,2013-01-01T13:00:00Z,2001.0,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,Turbo-fan,중거리,1
58,1,1,811.0,815,-4.0,1006.0,1032,-26.0,EV,4260,N11193,EWR,CHS,101.0,628,8,15,2013-01-01T13:00:00Z,2005.0,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,Turbo-fan,단거리,1


In [ ]:
ev.groupby('seats')[['dep_delay']]

seats
55    41658
80     7040
95      891
Name: count, dtype: int64

In [71]:
# 공항별 시트 좌석 수 파악
a = ev.groupby(['origin','dest', '거리구분','seats'])['운항_횟수'].sum()
a = a.reset_index()
a = a[a['운항_횟수'] != 0]
cond1 = (a['origin'] == 'EWR')
cond2 = (a['origin'] == 'LGA')
cond3 = (a['origin'] == 'JFK')

ewr_1 = a[cond1]
lga_1 = a[cond2]
jfk_1 = a[cond3]

C:\Users\USER\AppData\Local\Temp\ipykernel_2036\1007332264.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  a = ev.groupby(['origin','dest', '거리구분','seats'])['운항_횟수'].sum()


In [72]:
ewr_1

,origin,dest,거리구분,seats,운항_횟수
0,EWR,ALB,단거리,55,403
12,EWR,ATL,중거리,55,1200
13,EWR,ATL,중거리,80,200
14,EWR,ATL,중거리,95,210
18,EWR,AVL,단거리,55,245
27,EWR,BDL,단거리,55,388
57,EWR,BNA,중거리,55,1629
63,EWR,BOS,단거리,55,150
72,EWR,BTV,단거리,55,869
81,EWR,BUF,단거리,55,911


In [73]:
lga_1

,origin,dest,거리구분,seats,운항_횟수
1111,LGA,ATL,중거리,80,1
1135,LGA,BGR,단거리,80,335
1136,LGA,BGR,단거리,95,4
1147,LGA,BHM,중거리,80,241
1148,LGA,BHM,중거리,95,4
1156,LGA,BNA,중거리,80,339
1157,LGA,BNA,중거리,95,11
1170,LGA,BTV,단거리,55,2
1171,LGA,BTV,단거리,80,251
1172,LGA,BTV,단거리,95,1


In [74]:
jfk_1

,origin,dest,거리구분,seats,운항_횟수
562,JFK,ATL,중거리,80,1
598,JFK,BHM,중거리,80,1
783,JFK,IAD,단거리,55,1324
